# **Problem Statement**  
## **26. Implement a Skip List**

Implement a Skip List data structure supporting fast search, insertion, and deletion.
Skip lists use multiple levels of linked lists to provide average O(log n) time complexity for these operations.

### Constraints & Example Inputs/Outputs

- Keys are integers.
- Values can be integers or strings.
- No built-in ordered data structure (like TreeMap or sorted lists) can be used.

### Example 1:
```python
# Example Usage
skiplist = SkipList()
skiplist.insert(3, "apple")
skiplist.insert(7, "banana")
skiplist.insert(9, "orange")
print(skiplist.search(7))   # Output: "banana"
skiplist.remove(7)
print(skiplist.search(7))   # Output: None


### Solution Approach

What is a Skip List:

- A Skip List is a probabilistic data structure.
- Think of it as multiple linked lists layered on top of each other.
- Lower levels store all elements, higher levels store a subset.
- Faster search by skipping over large blocks.

Operations:

- Search: Start at highest level, move forward until a key larger than target is found, drop down a level, repeat until bottom.
- Insert: Search to find the insertion point, insert at level 0, decide how many levels to add probabilistically.
- Delete: Search for the node in all levels and remove.

Advantages:

- Average O(log n) for search, insert, delete.
- Simple to implement compared to balanced trees.

### Solution Code

For skip lists, a brute force approach is basically a simple sorted linked list.

In [1]:
# Approach1: Brute Force Approach
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class SkipListBrute:
    def __init__(self):
        self.head = None

    def insert(self, key, value):
        new_node = Node(key, value)
        if not self.head or key < self.head.key:
            new_node.next = self.head
            self.head = new_node
            return
        prev = None
        curr = self.head
        while curr and curr.key < key:
            prev = curr
            curr = curr.next
        if curr and curr.key == key:
            curr.value = value
            return
        new_node.next = curr
        prev.next = new_node

    def search(self, key):
        curr = self.head
        while curr:
            if curr.key == key:
                return curr.value
            curr = curr.next
        return None

    def remove(self, key):
        prev = None
        curr = self.head
        while curr:
            if curr.key == key:
                if prev:
                    prev.next = curr.next
                else:
                    self.head = curr.next
                return
            prev = curr
            curr = curr.next

### Alternative Solution

In [2]:
# Approach2: Optimized Approach (Skip List)
import random

class SkipListNode:
    def __init__(self, key, value, level):
        self.key = key
        self.value = value
        self.forward = [None] * (level + 1)

class SkipList:
    def __init__(self, max_level=4, p=0.5):
        self.max_level = max_level
        self.p = p
        self.header = SkipListNode(None, None, self.max_level)
        self.level = 0

    def random_level(self):
        lvl = 0
        while random.random() < self.p and lvl < self.max_level:
            lvl += 1
        return lvl

    def insert(self, key, value):
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        if current and current.key == key:
            current.value = value
        else:
            lvl = self.random_level()
            if lvl > self.level:
                for i in range(self.level + 1, lvl + 1):
                    update[i] = self.header
                self.level = lvl
            new_node = SkipListNode(key, value, lvl)
            for i in range(lvl + 1):
                new_node.forward[i] = update[i].forward[i]
                update[i].forward[i] = new_node

    def search(self, key):
        current = self.header
        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.key == key:
            return current.value
        return None

    def remove(self, key):
        update = [None] * (self.max_level + 1)
        current = self.header
        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current
        current = current.forward[0]
        if current and current.key == key:
            for i in range(self.level + 1):
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]
            while self.level > 0 and not self.header.forward[self.level]:
                self.level -= 1
    

### Test Cases 

In [3]:
def test_skip_list(cls):
    sl = cls()
    sl.insert(3, "apple")
    sl.insert(7, "banana")
    sl.insert(9, "orange")
    assert sl.search(7) == "banana"
    assert sl.search(4) is None
    sl.insert(7, "grape")
    assert sl.search(7) == "grape"
    sl.remove(7)
    assert sl.search(7) is None
    print(f"All test cases passed for {cls.__name__}!")

print("Testing Brute Force Skip List")
test_skip_list(SkipListBrute)

print("\nTesting Optimized Skip List")
test_skip_list(SkipList)


Testing Brute Force Skip List
All test cases passed for SkipListBrute!

Testing Optimized Skip List
All test cases passed for SkipList!


## Complexity Analysis

#### Time Complexity:

| Approach    | insert       | search       | remove       |
| ----------- | ------------ | ------------ | ------------ |
| Brute Force | O(n)         | O(n)         | O(n)         |
| Skip List   | O(log n) avg | O(log n) avg | O(log n) avg |

#### Space Complexity:
- O(n * log n) in Skip List due to multiple levels.

#### Thank You!!